In [11]:
import gym, os
from itertools import count
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical
import numpy as np

In [12]:
import gym.spaces as gs

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

"""class Wrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        self.observation_space = gs.Box(shape=(4,), low=-np.inf, high=np.inf)

    def observation(self, obs):
        t_0 = (obs[0]** 2 + obs[1]**2)**(0.5)
        t_1 = (obs[2]** 2 + obs[3]**2)**(0.5)
        t_2 = obs[4]/obs[5]
        t_3 = obs[6] or obs[7]
        return [t_0, t_1, t_2, t_3]"""
env = gym.make("LunarLander-v2")
#env = Wrapper(env)

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

In [14]:
import pennylane as qml

In [15]:
def ActorLayer(n_qubits, n_layers):
    dev = qml.device("default.qubit", wires=n_qubits)

    dimensions = {
        "x_weights": (n_layers, n_qubits),
        #"y_weights": (n_layers, n_qubits),
        "z_weights": (n_layers, n_qubits)
    }

    @qml.qnode(dev, interface='torch')
    def circuit(inputs, x_weights, z_weights):
        for layer_idx in range(n_layers):
            if (layer_idx == 0):
                for wire in range(n_qubits):
                    qml.RX(inputs[wire], wires=wire)
            for wire, x_weight in enumerate(x_weights[layer_idx]):
                qml.RX(x_weight, wires=wire)
            #for wire, y_weight in enumerate(y_weights[layer_idx]):
            #    qml.RX(y_weight, wires=wire)
            for wire, z_weight in enumerate(z_weights[layer_idx]):
                qml.RZ(z_weight, wires=wire)
            for wire in range(n_qubits-1):
                qml.CNOT(wires=[wire, (wire + 1) % n_qubits])
        return [
            qml.expval(qml.PauliZ(0) @ qml.PauliZ(1)),
            qml.expval(qml.PauliZ(2) @ qml.PauliZ(3)),
            qml.expval(qml.PauliZ(4) @ qml.PauliZ(5)),
            qml.expval(qml.PauliZ(6) @ qml.PauliZ(7)) 
        ]

    model = qml.qnn.TorchLayer(circuit, dimensions)
    return model

In [16]:
def CriticLayer(n_qubits, n_layers):
    dev = qml.device("default.qubit", wires=n_qubits)

    dimensions = {
        "x_weights": (n_layers, n_qubits),
        "y_weights": (n_layers, n_qubits),
        "z_weights": (n_layers, n_qubits)
    }

    @qml.qnode(dev, interface='torch')
    def circuit(inputs, x_weights, y_weights, z_weights):
        #print(inputs)
        for layer_idx in range(n_layers):
            if (layer_idx == 0):
                for wire in range(n_qubits):
                    qml.RX(inputs[wire], wires=wire)
            for wire, x_weight in enumerate(x_weights[layer_idx]):
                qml.RX(x_weight, wires=wire)
            for wire, y_weight in enumerate(y_weights[layer_idx]):
                qml.RX(y_weight, wires=wire)
            for wire, z_weight in enumerate(z_weights[layer_idx]):
                qml.RZ(z_weight, wires=wire)
            for wire in range(n_qubits):
                qml.CNOT(wires=[wire, (wire + 1) % n_qubits])
        return [
            qml.expval(qml.PauliZ(0) @ qml.PauliZ(1) @ qml.PauliZ(2) @ qml.PauliZ(3)),
        ]

    model = qml.qnn.TorchLayer(circuit, dimensions)
    return model

In [17]:
class Actor(nn.Module):
    def __init__(self, n_layers=5):
        super(Actor, self).__init__()
        self.n_qubits = 8
        self.n_actions = 4
        #self.data_reupload = data_reupload
        self.action_head = ActorLayer(n_qubits=self.n_qubits, n_layers=n_layers)

    def forward(self, state):
        state = torch.atan(state)
        outputs = self.action_head(state)
        outputs = -1 + (outputs - outputs.min()) * (2) / (outputs.max() - outputs.min())
        distribution = Categorical(F.softmax(outputs, dim=-1))
        return distribution


"""class Critic(nn.Module):
    def __init__(self, n_layers=4):
        super(Critic, self).__init__()
        self.n_qubits = 4
        self.n_actions = 4
        #self.data_reupload = data_reupload
        self.critic_head = CriticLayer(n_qubits=self.n_qubits, n_layers=n_layers)

    def forward(self, state):
        state = torch.atan(state)
        outputs = self.critic_head(state)
        return outputs"""

class Critic(nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = nn.Linear(self.state_size, 128)
        self.linear2 = nn.Linear(128, 256)
        self.linear3 = nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

In [18]:
def compute_returns(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns



In [19]:

def trainIters(actor, critic, n_iters):
    optimizerA = optim.Adam(actor.parameters(), lr = 0.005, amsgrad = True)
    optimizerC = optim.Adam(critic.parameters(), lr = 0.005, amsgrad = True)
    for iter in range(n_iters):
        state, _ = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()
        total_rewards = 0
        done = False

        while not done:
            #env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, term, trunc, _ = env.step(action.cpu().numpy())
            done = term or trunc

            total_rewards += reward
            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, total_rewards))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = compute_returns(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()
    torch.save(actor.state_dict(), 'actorql2.pkl')
    torch.save(critic.state_dict(), 'criticql2.pkl')
    env.close()



In [20]:

if __name__ == '__main__':
    if os.path.exists('quantum/actorql2.pkl'):
        actor = torch.load('quantum/actorql2.pkl')
        print('Actor Model loaded')
    else:
        actor = Actor().to(device)
    if os.path.exists('quantum/criticql2.pkl'):
        critic = torch.load('quantum/criticql2.pkl')
        print('Critic Model loaded')
    else:
        critic = Critic(state_size, action_size).to(device)
    trainIters(actor, critic, n_iters=1000)

Iteration: 0, Score: -450.79262441122
Iteration: 1, Score: -552.4075435834848
Iteration: 2, Score: -346.52786094315115
Iteration: 3, Score: -628.9079922247943
Iteration: 4, Score: -544.809247548179
Iteration: 5, Score: -474.21622918453465
Iteration: 6, Score: -276.7454870409143
Iteration: 7, Score: -642.6297978674203
Iteration: 8, Score: -390.0817240655901
Iteration: 9, Score: -399.67754559911884
Iteration: 10, Score: -564.0242868517032
Iteration: 11, Score: -445.5022784832256
Iteration: 12, Score: -212.21403286907798
Iteration: 13, Score: -665.8247136233491
Iteration: 14, Score: -746.1154561394575
Iteration: 15, Score: -421.1585739144139
Iteration: 16, Score: -565.3889685701828
Iteration: 17, Score: -407.72682741550335
Iteration: 18, Score: -1541.62285918527
Iteration: 19, Score: -653.8227452120342
Iteration: 20, Score: -427.78131175450795
Iteration: 21, Score: -54.33585781096552
Iteration: 22, Score: -43.55708674906111
Iteration: 23, Score: -341.7607717616431
Iteration: 24, Score: -1

In [25]:
new_env = gym.make("LunarLander-v2", render_mode = "human")

In [36]:
rewards = []

for i in range(10):
    done = False
    t_rewards = 0
    state, info = new_env.reset()
    while not done:
        state = torch.FloatTensor(state).to(device)
        dist, value = actor(state), critic(state)
        
        action = dist.sample()
        obs, reward, term, trunc, info = new_env.step(action.cpu().numpy())
        done = term or trunc

        t_rewards += reward
        if done:
            break
        else:
            state = obs
    
    rewards.append(t_rewards)
    print(f"Episode {i}: Reward {t_rewards}")
new_env.close()

Episode 0: Reward -49.78552192988293
Episode 1: Reward 2.2276536249626275
Episode 2: Reward 28.76865084190905
Episode 3: Reward -9.317536109760383
Episode 4: Reward 6.230408887518408
Episode 5: Reward -79.24190120730574
Episode 6: Reward -40.12709242070673
Episode 7: Reward -18.097244358670906
Episode 8: Reward -20.540650554761058
Episode 9: Reward 50.8625563142917


In [38]:
import imageio

def saveanimation(frames,address="./a2cll.gif"):
    imageio.mimwrite(address, frames)

In [39]:
env1 = gym.make("LunarLander-v2", render_mode = "rgb_array")

In [40]:
rewards = []
frames = []

for i in range(5):
    done = False
    t_rewards = 0
    state, info = env1.reset()
    while not done:
        frames.append(env1.render())
        state = torch.FloatTensor(state).to(device)
        dist, value = actor(state), critic(state)
        
        action = dist.sample()
        obs, reward, term, trunc, info = env1.step(action.cpu().numpy())
        done = term or trunc

        t_rewards += reward
        if done:
            break
        else:
            state = obs
    
    rewards.append(t_rewards)
    print(f"Episode {i}: Reward {t_rewards}")

saveanimation(frames)
env1.close()

Episode 0: Reward 1.2423838147589379
Episode 1: Reward -56.596468678655484
Episode 2: Reward -25.2634833339704
Episode 3: Reward -41.0191044209007
Episode 4: Reward -83.97540137624948


In [42]:
torch.save(actor.state_dict(), "a2ca.pt")
torch.save(critic.state_dict(), "a2cc.pt")
